In [1]:
%load_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
print('load data')
path = '../../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

load data


In [2]:
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy

def get_pdbqt(mol):

    # add hydrogens (without regard to pH)
    protonated_mol = rdkit.Chem.AddHs(mol)

    # generate 3D coordinates for the ligand. 
    rdkit.Chem.AllChem.EmbedMolecule(protonated_mol)

    # intialize preparation
    preparator = MoleculePreparation()
    mol_setups = preparator.prepare(protonated_mol)
    
    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            return pdbqt_string

Create ligand PDBQTs.

In [3]:
import os
from tqdm.notebook import tqdm

out_dir = '../../dgym-data/out/ligands_temp/'

paths = []
for mol in tqdm(deck[-96:]):
    path = os.path.join(out_dir, f'{mol.name}.pdbqt')
    paths.append(path)
    pdbqt = get_pdbqt(mol.mol)
    
    with open(path, 'w') as file:
        file.write(pdbqt)

ligands_txt = ' '.join(paths)
path = os.path.join(out_dir, 'ligands.txt')
with open(path, 'w') as file:
    file.write(ligands_txt)

  0%|          | 0/96 [00:00<?, ?it/s]

Create ligand txt summary file.

In [4]:
# import subprocess
# import tempfile

# with tempfile.TemporaryDirectory() as tempdir:
#     print(f"Temporary directory created at {tempdir}")

In [5]:
import subprocess

command = 'unidock --receptor ../../dgym-data/Mpro_prepped.pdbqt --ligand_index ../../dgym-data/out/ligands_temp/ligands.txt --center_x 9.812 --center_y -0.257 --center_z 20.8485 --size_x 14.328 --size_y 8.85 --size_z 12.539 --dir ../../dgym-data/out --exhaustiveness 128 --max_step 20 --num_modes 9 --scoring vina --refine_step 3 --seed 5'
resp = subprocess.run(
    command,
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE, 
    encoding='utf-8'
)

In [14]:
import glob
from itertools import islice
from operator import itemgetter

affinities = []
paths = glob.glob('../../dgym-data/out/*.pdbqt')
for path in paths:
    with open(path, 'r') as file:
        # get relevant lines from file
        lines = list(islice(file, 7))
        affinity_str, smiles_str = itemgetter(1, 6)(lines)
        
        # extract affinity
        affinity = affinity_str.split('    ')[1].split('    ')[0]
        smiles = smiles_str.split(' ')[-1].split('\n')[0]

        # append to affinities
        affinities.append(
            {'smiles': smiles, 'affinity': float(affinity)}
        )

In [20]:
import numpy as np

def boltzmann_sum(energies, temperature=298.15):
    kT = 0.0019872041 * temperature  # Boltzmann constant in kcal/(mol·K) multiplied by temperature in K
    energies = np.array(energies)  # Energies should be in kcal/mol

    # Use logsumexp for numerical stability
    return -kT * np.log(np.sum(np.exp(-energies / kT)))

In [38]:
import glob
from itertools import islice
from operator import itemgetter

affinities = []
paths = glob.glob('../../dgym-data/out/*.pdbqt')

for path in paths:
    with open(path, 'r') as file:
        
        # get relevant lines from file
        smiles_str = list(islice(file, 7))[-1]
        smiles = smiles_str.split(' ')[-1].split('\n')[0]

        # extract affinities
        affinity_strs = [line for line in file
                         if line.startswith('REMARK VINA RESULT')]
        process_affinity = lambda s: s.split('    ')[1].split('    ')[0]
        affinity_scores = [float(process_affinity(a)) for a in affinity_strs]
        
        # compute boltzmann sum
        affinity = boltzmann_sum(affinity_scores)

        # append to affinities
        affinities.append(
            {'smiles': smiles, 'affinity': affinity}
        )

affinities = pd.DataFrame(affinities)

In [39]:
import pandas as pd

affinities.sort_values('affinity')

,smiles,affinity
1,Cc1cccc(Nc2ncnc3c2cnn3C)c1,-7.453299
56,Cc1ccc(NC(=O)c2cccc(F)c2)cc1O,-7.231530
19,O=C(NCCc1ccncc1)c1ccccc1F,-7.194071
6,Oc1ncc(NCCc2ccccc2)c(O)n1,-7.167499
82,O=C(NCCc1ccc(O)cc1)c1ccccn1,-7.161006
...,...,...
84,CNC(C)c1cnn(C)c1,-5.039811
88,CN(C)Cc1nc(C2(N)CCCC2)no1.Cl.Cl,-1.914013
44,CS(=O)(=O)Nc1ccc(CN)cc1.Cl,-1.912712
78,CS(=O)(=O)NC1CCNCC1.Cl,-1.908687
